<a href="https://colab.research.google.com/github/shevandrin/SSEresearch/blob/main/ESE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h3> variables for research:</h3>

|#|variable|code of variable|accessible|
|-|--------|----------------|-|
|1|language community|language|-|
|2|type of error|error_type|-|
|3|time stamp|time|+|
|4|associated tags|tags|+|
|5|number of comments|comments|+|
|6|number of answers|answers|+|
|7|numver of views|views|+|
|8|number of votes|votes|+|
|9|correctness of answer|<b><i>correctness</i></b>|+|
|10|data of accquisitioin|<b><i>date</i></b>|-|

<b><i>new variables that were not put in the first milestone report</i></b>

types of error see here: https://www.tutorialsteacher.com/python/error-types-in-python

Two phases algorithm:
* 0. getting labels of Python errors
* 1. collect questions id for each type of error (precise search phrase might look like 'title:"AssertionError" [Python] is:question')
* 2. collect metadata from each error page (example of queston page url: https://stackoverflow.com/questions/11181519/)

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

<h3><span style="color:red">Phase 0</span></h3>
getting types of Python errors<br>
<b>!!! the data is already got, this block can be missed</b>

In [ ]:
page_with_python_errors = "https://www.tutorialsteacher.com/python/error-types-in-python"
res = requests.get(page_with_python_errors)
soup = BeautifulSoup(res.text, "html.parser")

In [ ]:
table_rows = soup.find_all("tr")
l = []
for row in table_rows[1:]:
  cell = row.find_all("td")
  result = {}
  result['code_error'] =  cell[0].text.strip()
  result['description'] = cell[1].text.strip()
  l.append(result)
df = pd.DataFrame.from_dict(l)
df.to_csv("python_error_types.csv")
df.head()

,code_error,description
0,AssertionError,Raised when the assert statement fails.
1,AttributeError,Raised on the attribute assignment or referenc...
2,EOFError,Raised when the input() function hits the end-...
3,FloatingPointError,Raised when a floating point operation fails.
4,GeneratorExit,Raised when a generator's close() method is ca...


Now types of python errors are accessible from here:

In [ ]:
url = 'https://raw.githubusercontent.com/bondarenko-petr/ewe2/main/python_error_types.csv'
df1 = pd.read_csv(url)
df1.head(2)

,Unnamed: 0,code_error,description
0,0,AssertionError,Raised when the assert statement fails.
1,1,AttributeError,Raised on the attribute assignment or referenc...


<h3><span style="color:red">Phase 2</span></h3>
getting ids of questions and other attributes from question list page

In [ ]:
url0 = "https://stackoverflow.com/questions"
url1 = "https://stackoverflow.com/questions/17256602"
url2 = "https://stackoverflow.com/questions/71792649"

In [ ]:
def get_page_data(url):
  res = requests.get(url)
  soup = BeautifulSoup(res.text, "html.parser")
  # for number of vies we need to extract container with vies and rid of label
  question_block = soup.find("div", attrs={"class":"flex--item ws-nowrap mb8"}).next.next.next.next.strip()
  pos = question_block.find(" ")
  # for time stamp we need the attribute 'datetime' of the first tag 'time'
  creation_stamp = soup.find("time")["datetime"]
  votes = soup.find("div", attrs={"class":"js-vote-count flex--item d-flex fd-column ai-center fc-black-500 fs-title"})["data-value"]
  answers = soup.find("h2", attrs={"class":"mb0"})["data-answercount"]
  tags = soup.find("ul", attrs={"class":"ml0 list-ls-none js-post-tag-list-wrapper d-inline"}).find_all("li")
  tags = list(map(lambda x: x.text, tags))
  solved = soup.find("div", attrs={"itemprop":"acceptedAnswer"})
  solved = "unsolved" if solved is None else "solved"
  comments =  len(soup.find_all("span", attrs={"class":"comment-copy"}))
  res = {}
  res['time'] =  creation_stamp
  res["tags"] =  tags
  res["comments"] = comments
  res["answers"] = answers
  res["views"] = question_block[:pos]
  res["votes"] = votes
  res["correctness"] = solved
  
  return res


In [ ]:
get_page_data(url2)

{'time': '2022-04-08T06:37:23',
 'tags': ['python', 'pytorch', 'attention-model'],
 'comments': 0,
 'answers': '0',
 'views': '229',
 'votes': '0',
 'correctness': 'unsolved'}